In [1]:
from csv import reader
import numpy as np
from unidecode import unidecode
import pandas as pd
import statsmodels.api as sm


#### Nesse teste de hipótese estariamos avaliando qual a influência das mulheres nas votações, será que os homens possuem além de maior representativade maior facilidade para aprovação de propostas?

In [4]:
deputados = pd.read_csv('ArquivosLimpos/deputados-2015-2018.csv').dropna()

In [5]:
deputados.dtypes

deputado_id                 int64
nome                       object
idLegislaturaInicial        int64
idLegislaturaFinal          int64
siglaSexo                  object
betweenness_centrality    float64
eigenvecto_centrality     float64
dtype: object

In [7]:
votacoes = pd.read_csv('ArquivosLimpos/votacoes-2015-2018.csv').dropna()

In [8]:
votacoes.dtypes

id                                            object
idOrgao                                        int64
idEvento                                       int64
aprovacao                                    float64
votosSim                                       int64
votosNao                                       int64
votosOutros                                    int64
ultimaApresentacaoProposicao_idProposicao      int64
dtype: object

In [10]:
votos = pd.read_csv('ArquivosLimpos/votacoesVotos-2015-2018.csv').dropna()

In [162]:
votos.dtypes

Unnamed: 0                int64
idVotacao                object
voto                      int64
deputado_id               int64
deputado_siglaPartido    object
pardido_id                int64
deputado_siglaUf         object
dtype: object

In [11]:
proposicoes = pd.read_csv('ArquivosLimpos/proposicoesAutores-2015-2018.csv').dropna()

In [12]:
data_dep_prop = pd.merge(deputados, proposicoes, left_on='deputado_id', right_on='idDeputadoAutor')

In [13]:
data_voto_vota = pd.merge(votacoes,votos, left_on='id', right_on='idVotacao')

In [14]:
 data= pd.merge(data_dep_prop,votacoes, left_on='idProposicao', right_on='ultimaApresentacaoProposicao_idProposicao')

In [15]:
data_dep_prop.head()


,deputado_id,nome,idLegislaturaInicial,idLegislaturaFinal,siglaSexo,betweenness_centrality,eigenvecto_centrality,idProposicao,idDeputadoAutor
0,66179,Norma Ayub,55,56,F,0.0,3.907493e-11,2075791,66179.0
1,66179,Norma Ayub,55,56,F,0.0,3.907493e-11,1594145,66179.0
2,66179,Norma Ayub,55,56,F,0.0,3.907493e-11,1527018,66179.0
3,66179,Norma Ayub,55,56,F,0.0,3.907493e-11,1278875,66179.0
4,66179,Norma Ayub,55,56,F,0.0,3.907493e-11,1230242,66179.0


In [16]:
data.head()

,deputado_id,nome,idLegislaturaInicial,idLegislaturaFinal,siglaSexo,betweenness_centrality,eigenvecto_centrality,idProposicao,idDeputadoAutor,id,idOrgao,idEvento,aprovacao,votosSim,votosNao,votosOutros,ultimaApresentacaoProposicao_idProposicao
0,66828,Fausto Pinato,55,56,M,0.0,3.907493e-11,2056626,66828.0,521407-108,2003,46088,0.0,0,0,0,2056626
1,66828,Fausto Pinato,55,56,M,0.0,3.907493e-11,2056626,66828.0,521407-123,2003,46230,1.0,0,0,0,2056626
2,66828,Fausto Pinato,55,56,M,0.0,3.907493e-11,1738718,66828.0,402642-124,2003,41191,1.0,0,1,0,1738718
3,66828,Fausto Pinato,55,56,M,0.0,3.907493e-11,1738718,66828.0,402642-144,2003,42072,1.0,0,0,0,1738718
4,66828,Fausto Pinato,55,56,M,0.0,3.907493e-11,1514542,66828.0,431099-141,2003,40265,1.0,0,0,0,1514542


In [17]:
data1= data.idProposicao.value_counts().head(5)

In [18]:
data2= data.aprovacao.value_counts().head(5)

In [20]:
values = deputados['siglaSexo'].unique()

for value in values[:-1]:
    deputados[value] = [1 if v == value else 0 for v in deputados['siglaSexo']]

In [21]:
deputados[values[:-1]]

,F
0,1
1,0
2,1
3,0
4,0
...,...
267,0
268,0
269,0
270,0


#### Influênca sexo na influência de um deputado

In [22]:
model = sm.Logit(deputados['eigenvecto_centrality'], deputados[values[:-1]])
result = model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.609767
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                             Logit Regression Results                            
=================================================================================
Dep. Variable:     eigenvecto_centrality   No. Observations:                  272
Model:                             Logit   Df Residuals:                      271
Method:                              MLE   Df Model:                            0
Date:                   Fri, 13 Nov 2020   Pseudo R-squ.:                     inf
Time:                           19:19:52   Log-Likelihood:                -165.86
converged:                          True   LL-Null:                        0.0000
Covariance Type:               nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
F             -5.2196      2.380     -2.193      0.028      -9.883      -0.556
==============================================================================
"""

#### Influência eficiência na inflência de um deputado